In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 723.1/723.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [4]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import warnings
warnings.simplefilter("ignore")
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
from ultralytics import YOLO

model = YOLO('yolov8n-pose.pt')

100%|██████████| 6.51M/6.51M [00:00<00:00, 56.9MB/s]


In [19]:
def significant_pose_detection(arr, th):
  n = len(arr)
  i=0
  for itemm in arr:
    if np.sum(itemm[0]+itemm[1])!=0:
      i = i + 1

  sig_det = (i/n)
  if sig_det>=th:
    return 1

  return 0
def process_video(input_video_path, output_video_path):
    # Open the video file
    video = cv2.VideoCapture(input_video_path)

    # Get the video's frame rate
    fps = video.get(cv2.CAP_PROP_FPS)

    # Get the video's width and height
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create a VideoWriter object to write processed frames to a new video file
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    while True:
        # Read the next frame from the video
        ret, frame = video.read()

        # If the frame was not successfully read, then we have reached the end of the video
        if not ret:
            break

        # Process the frame here...
        results = model.predict(frame)[0]
        th = 0.3
        Q = []
        det = 0
        concat_df = pd.DataFrame()

        for result in results:
            landmarks = []
            kpts = result.keypoints
            nk = kpts.shape[1]

            for i in range(nk):
                keypoint = kpts.xy[0, i]
                x, y = int(keypoint[0].item()), int(keypoint[1].item())
                landmarks.append([x,y])

            detections = significant_pose_detection(landmarks, th)
            if detections==1:
                det = det + 1
                for j in range(len(landmarks)):
                    x = landmarks[j][0]
                    y = landmarks[j][1]
                    # cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)
                    ttext = "("+str(x)+","+str(y)+")"
                    # cv2.putText(frame, ttext, (x, y), cv2.FONT_HERSHEY_SIMPLEX,0.3, (0,0,0), 1, cv2.LINE_AA)

                df = pd.DataFrame(landmarks, columns =['x'+str(det), 'y'+str(det)], dtype = float)
                concat_df = pd.concat([concat_df, df], axis=1)

                # cv2.line(frame, landmarks[11], landmarks[12], (0,255,0), 2)
                a = (landmarks[11][0] + landmarks[12][0])//2
                b = (landmarks[11][1] + landmarks[12][1])//2
                Q.append([a,b])

        m = list(map(lambda l: [v for v in l if v != 0], Q))
        nQ = [ele for ele in m if ele != []]

        k = len(nQ)

        i=0
        while i<(k-1):
            # cv2.line(frame, nQ[i], nQ[i+1], (255, 0,0), 2)
            i = i + 1

        i = 0
        while i<k:
            # cv2.circle(frame, nQ[i], 5, (0, 255, 255), -1)
            # cv2.circle(frame, nQ[i], 7, (0, 0, 255), 2)
            i = i + 1

        concat_df.to_csv('detected_pose.csv')
        df_nQ = pd.DataFrame(nQ, columns =['x', 'y'], dtype = int)
        df_nQ.to_csv('nQ_centroids.csv')

        # Perform linear regression and outlier detection
        url = 'nQ_centroids.csv'
        data = pd.read_csv(url)
        x = data['x'].values
        y = data['y'].values

        slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

        y_pred = intercept + slope * x
        residuals = y - y_pred

        residuals_std = np.std(residuals)

        ci_lower = y_pred - 1.440*residuals_std
        ci_upper = y_pred + 1.440*residuals_std

        outliers = np.where((y < ci_lower) | (y > ci_upper))

        print("Outliers: ", outliers)


        # persons = concat_df.values.reshape(-1, 8, 2)
        persons=concat_df.values

        for i in outliers[0]:
            x_coords = concat_df['x'+str(i+1)].dropna().values
            y_coords = concat_df['y'+str(i+1)].dropna().values
            person = np.column_stack((x_coords, y_coords))
            person = person[~np.all(person == [0, 0], axis=1)]
            x_min, y_min = np.min(person, axis=0)
            x_max, y_max = np.max(person, axis=0)
            cv2.rectangle(frame, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0,0,255), 2)

        # Write the processed frame to the new video file
        out.write(frame)

    # Release the VideoCapture and VideoWriter objects
    video.release()
    out.release()

# Call the function with your input and output video paths
process_video('people_in_queue.mp4', 'outlier_video.mp4')


0: 384x640 11 persons, 190.8ms
Speed: 4.8ms preprocess, 190.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Outliers:  (array([4, 9]),)

0: 384x640 11 persons, 142.0ms
Speed: 2.7ms preprocess, 142.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Outliers:  (array([5, 9]),)

0: 384x640 11 persons, 139.5ms
Speed: 2.3ms preprocess, 139.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Outliers:  (array([6, 9]),)

0: 384x640 11 persons, 138.7ms
Speed: 2.3ms preprocess, 138.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Outliers:  (array([6, 9]),)

0: 384x640 11 persons, 154.3ms
Speed: 2.4ms preprocess, 154.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Outliers:  (array([7, 9]),)

0: 384x640 11 persons, 145.8ms
Speed: 2.1ms preprocess, 145.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Outliers:  (array([7, 9]),)

0: 384x640 11 persons, 153.1ms
Speed: 3.8ms preproc

In [21]:
def significant_pose_detection(arr, th):
  n = len(arr)
  i=0
  for itemm in arr:
    if np.sum(itemm[0]+itemm[1])!=0:
      i = i + 1

  sig_det = (i/n)
  if sig_det>=th:
    return 1
  return 0

def hconcat_resize(img_list,  interpolation  = cv2.INTER_CUBIC):
    h_min = min(img.shape[0]  for img in img_list)
    im_list_resize = [cv2.resize(img, (int(img.shape[1] * h_min / img.shape[0]), h_min), interpolation = interpolation)  for img in img_list]
    return cv2.hconcat(im_list_resize)

def process_video(input_video_path, output_video_path):
    video = cv2.VideoCapture(input_video_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
    while True:
        ret, frame = video.read()
        if not ret:
            break
        results = model.predict(frame)[0]
        th = 0.3
        Q = []
        det = 0
        concat_df = pd.DataFrame()
        for result in results:
            landmarks = []
            kpts = result.keypoints
            nk = kpts.shape[1]
            for i in range(nk):
                keypoint = kpts.xy[0, i]
                x, y = int(keypoint[0].item()), int(keypoint[1].item())
                landmarks.append([x,y])
            detections = significant_pose_detection(landmarks, th)
            if detections==1:
                det = det + 1
                for j in range(len(landmarks)):
                    x = landmarks[j][0]
                    y = landmarks[j][1]
                    # cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)
                    ttext = "("+str(x)+","+str(y)+")"
                    # cv2.putText(frame, ttext, (x, y), cv2.FONT_HERSHEY_SIMPLEX,0.3, (0,0,0), 1, cv2.LINE_AA)
                df = pd.DataFrame(landmarks, columns =['x'+str(det), 'y'+str(det)], dtype = float)
                concat_df = pd.concat([concat_df, df], axis=1)
                # cv2.line(frame, landmarks[11], landmarks[12], (0,255,0), 2)
                a = (landmarks[11][0] + landmarks[12][0])//2
                b = (landmarks[11][1] + landmarks[12][1])//2
                Q.append([a,b])

        dict={}
        sorted_Q=sorted(Q)
        for i,point in enumerate(Q):
          j=sorted_Q.index(point)
          dict[j]=i
        m = list(map(lambda l: [v for v in l if v != 0], sorted(Q)))
        nQ = [ele for ele in m if ele != []]
        k = len(nQ)
        h,w,c = frame.shape
        canva = np.zeros([h,w,3],dtype=np.uint8)
        nQT = [[itemm[1],itemm[0]] for itemm in nQ]
        i=0
        while i<(k-1):
            cv2.line(canva, nQ[i], nQ[i+1], (255, 0,0), 2)
            i = i + 1
        i = 0
        while i<k:
            cv2.circle(canva, nQ[i], 5, (0, 255, 255), -1)
            cv2.circle(canva, nQ[i], 7, (0, 0, 255), 2)
            i = i + 1
        canva = cv2.rotate(canva, cv2.ROTATE_90_CLOCKWISE)
        image = cv2.flip(canva, 1)
        i = 0
        while i<k:
          ttext = "("+str(nQT[i][0])+","+str(nQT[i][1])+")"
          cv2.putText(image, ttext, (nQ[i][1]+15, nQ[i][0]), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,255), 1, cv2.LINE_AA)
          cv2.putText(image, "Person_"+str(i+1), (nQT[i][0]-55, nQT[i][1]), cv2.FONT_HERSHEY_SIMPLEX,
                            0.3, (0,255,0), 1, cv2.LINE_AA)
          i = i + 1
        cv2.imwrite("canvas.jpg", image)
        concat_df.to_csv('detected_pose.csv')
        df_nQ = pd.DataFrame(nQ, columns =['x', 'y'], dtype = int)
        df_nQ.to_csv('nQ_centroids.csv')
        url = 'nQ_centroids.csv'
        data = pd.read_csv(url)
        x = data['x'].values
        y = data['y'].values
        slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
        y_pred = intercept + slope * x
        residuals = y - y_pred
        residuals_std = np.std(residuals)
        ci_lower = y_pred - 1.440*residuals_std
        ci_upper = y_pred + 1.440*residuals_std
        outliers = np.where((y < ci_lower) | (y > ci_upper))
        print("Outliers: ", outliers)
        # persons = concat_df.values.reshape(-1, 8, 2)
        persons=concat_df.values

        for i in outliers[0]:
            x_coords = concat_df['x'+str(dict[i]+1)].dropna().values
            y_coords = concat_df['y'+str(dict[i]+1)].dropna().values
            person = np.column_stack((x_coords, y_coords))
            person = person[~np.all(person == [0, 0], axis=1)]
            x_min, y_min = np.min(person, axis=0)
            x_max, y_max = np.max(person, axis=0)
            cv2.rectangle(frame, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0,0,255), 2)

        # doubt
        # x, y, w1, h1 = w - nQT[0][0] - 25, nQT[0][1] - 20, 50, nQT[k - 1][1] - nQT[0][1] + 40
        # print(x,y,w1,h1)
        # cv2.rectangle(image, (x,y), (x+w1,y+h1), (0, 200, 0), -1)
        # cv2.imwrite("canvas1.jpg", image)
        cv2.imwrite("frame.jpg", frame)
        frame_img=cv2.imread('frame.jpg')
        img_h_resize = hconcat_resize([frame_img, image])
        # cv2_imshow(img_h_resize)
        out.write(img_h_resize)

    video.release()
    out.release()

process_video('people_in_queue.mp4', 'output_video.mp4')


0: 384x640 11 persons, 185.9ms
Speed: 2.5ms preprocess, 185.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Outliers:  (array([5, 9]),)

0: 384x640 11 persons, 143.3ms
Speed: 4.0ms preprocess, 143.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Outliers:  (array([5, 9]),)

0: 384x640 11 persons, 145.8ms
Speed: 4.5ms preprocess, 145.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Outliers:  (array([5, 9]),)

0: 384x640 11 persons, 143.5ms
Speed: 2.2ms preprocess, 143.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Outliers:  (array([5, 9]),)

0: 384x640 11 persons, 145.6ms
Speed: 3.7ms preprocess, 145.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Outliers:  (array([5, 9]),)

0: 384x640 11 persons, 144.8ms
Speed: 3.8ms preprocess, 144.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Outliers:  (array([5, 9]),)

0: 384x640 11 persons, 144.4ms
Speed: 3.0ms preproc